In [1]:
import sys
sys.path.append("..")

import os

In [2]:
import os
import tempfile
import datetime
from ruamel.yaml import YAML
import json

import source.helpers.general as helpers
from source.helpers.types import StrPath

# Import of Pipeline Steps
from source.conversion.msconv_pipe import File_Converter
from source.feature_finding.mzmine_pipe import MZmine_Runner
from source.annotation.sirius_pipe import Sirius_Runner
from source.annotation.gnps_pipe import GNPS_Runner

class MS_Analysis_Configuration:
    def __init__( self, platform:str="Linux", overwrite:bool=False, nested:bool=False,
                  save_log:bool=True, verbosity:int=1,
                  file_converter:File_Converter|dict=File_Converter(),
                  mzmine_runner:MZmine_Runner|dict=MZmine_Runner(),
                  gnps_runner:GNPS_Runner|dict=GNPS_Runner(),
                  sirius_runner:Sirius_Runner|dict=Sirius_Runner() ):
        self.platform       = platform
        self.overwrite      = overwrite
        self.nested         = nested
        self.save_log       = save_log
        self.verbosity      = verbosity
        self.file_converter = File_Converter(**file_converter)  if isinstance(file_converter, dict) else file_converter
        self.mzmine_runner  = MZmine_Runner(**mzmine_runner)    if isinstance(mzmine_runner, dict)  else mzmine_runner
        self.gnps_runner    = GNPS_Runner(**gnps_runner)        if isinstance(gnps_runner, dict)    else gnps_runner
        self.sirius_runner  = Sirius_Runner(**sirius_runner)    if isinstance(sirius_runner, dict)  else sirius_runner
                  

    def update( self, dictionary:dict=None, **kwargs ):
        if not dictionary:
            dictionary = kwargs
        for key, value in dictionary.items():
            setattr( self, key, value )

    def make_nested_attributes_dict( self, object=None, attributes_dict:dict={}):
        if not object:
            object = self
        for attribute, value in object.__dict__.items():
            if hasattr(value, "__dict__"):
                attributes_dict[attribute] = self.make_nested_attributes_dict( object=value, attributes_dict={} )
            else:
                attributes_dict[attribute] = value
        return attributes_dict

    def dict_representation( self, attribute ):
        attributes_dict = {}
        for attribute, value in attribute.__dict__.items():
            if hasattr(value, "__dict__"):
                attributes_dict[attribute] = self.dict_representation(value)
            else:
                attributes_dict[attribute] = value
        return attributes_dict
    
    def save( self, location ):
        yaml = YAML()
        with open( location, "w") as f:
            yaml.dump( self.dict_representation( self ), f )

    def load( self, location ):
        yaml = YAML()
        with open( location, "r") as f:
            config = yaml.load( f.read() )
        self.update( **config )

ms_analysis_configuration = MS_Analysis_Configuration()
ms_analysis_configuration.save( "test.yaml" )

In [3]:
ms_analysis_configuration.load( "test.yaml" )

In [4]:
yaml = YAML()
with open( "test.yaml", "r") as f:
    config = yaml.load( f.read() )
msac = MS_Analysis_Configuration( **config )

TypeError: source.helpers.classes.Pipe_Step.__init__() got multiple values for keyword argument 'patterns'

In [ ]:
loaded_dict

'test.yaml'